In [1]:
# Import libraries
import cudf
import pandas as pd
from numba import cuda
import shapefile as sh
import numpy as np
import math

In [2]:
# Read in US Tiger Line Edge shapefile (Example here is King County)
shp_name = "/data/us_tiger/king_county/tl_2018_06085_edges.shp"
sf = sh.Reader(shp_name)

In [3]:
# Get field positions for road type and nodes
fields = list(zip(*sf.fields))
mtfcc_pos = fields[0].index('MTFCC') - 1
roadflg_pos = fields[0].index('ROADFLG') - 1
nodef_pos = fields[0].index('TNIDF') - 1
nodet_pos = fields[0].index('TNIDT') - 1

In [4]:
# Parse shape file into list of edges and their constituent vectors
vectors = []
edges = []
# Read nodes into a dictionary to de-duplicate
nodes = {}
node_num = 0
mtfcc = []
# Iterate through shapefile shape objects
for i,geo in enumerate(sf.shapes()):
    # Get the shape attributes
    attr = sf.record(i)
    # Check that the edge is a road
    if attr[roadflg_pos] == 'Y':
        # Test if mtfcc (edge classification) is in list, if not add it
        if attr[mtfcc_pos] not in mtfcc:
            mtfcc += [attr[mtfcc_pos]]
        # Test if from node exists
        if attr[nodef_pos] not in nodes:
            # If not, add it to the dictionary with coordinates of first point of vector
            nodes[attr[nodef_pos]] = [node_num,geo.points[0][0],geo.points[0][1]]
            node_num += 1
        # Test if to node exists
        if attr[nodet_pos] not in nodes:
            # If not, add it to the dictionary with coordinates of last point of vector
            nodes[attr[nodet_pos]] = [node_num,geo.points[-1][0],geo.points[-1][1]]
            node_num += 1
        # Add edge to table 
        edges += [[i,nodes[attr[nodef_pos]][0],nodes[attr[nodet_pos]][0],attr[mtfcc_pos]]]
        # Add edge to table with from and too nodes reversed
        edges += [[i,nodes[attr[nodet_pos]][0],nodes[attr[nodef_pos]][0],attr[mtfcc_pos]]]
        # Unravel and serialise the vector points into a list
        for p in geo.points:
            vectors += [[i,p[0],p[1]]]
# Sort mtfcc
mtfcc.sort()

In [5]:
# Zip the vector points and copy to cudf DataFrame
vz = list(zip(*vectors))
vector_df = cudf.DataFrame()
vector_df['id'] = vz[0]
vector_df['lon'] = vz[1]
vector_df['lat'] = vz[2]

In [6]:
# Zip the node points and copy to cudf DataFrame
nz = list(zip(*sorted(nodes.values())))
node_df = cudf.DataFrame()
node_df['id'] = nz[0]
node_df['lon'] = nz[1]
node_df['lat'] = nz[2]

In [7]:
# Zip the edges and copy to cudf DataFrame
ez = list(zip(*edges))
edge_df = cudf.DataFrame()
edge_df['id'] = ez[0]
edge_df['src'] = ez[1]
edge_df['dst'] = ez[2]
edge_df['mtfcc'] = [mtfcc.index(m) for m in ez[3]]

In [8]:
# Check results on vector Dataframe
print(vector_df)

            id         lon        lat
0            1 -121.606408  37.191532
1            1 -121.606128  37.191692
2            1 -121.605951  37.191783
3            1 -121.605861  37.191814
4            1 -121.605767  37.191832
5            1 -121.605660  37.191836
6            1 -121.605483  37.191840
7            1 -121.605306  37.191857
8            1 -121.605152  37.191881
9            1 -121.604765  37.191944
10           1 -121.604598  37.191971
11           1 -121.604463  37.191984
12           1 -121.604327  37.191989
13           1 -121.604044  37.191988
14           1 -121.603922  37.192002
15           1 -121.603800  37.192030
16           1 -121.603692  37.192068
17           1 -121.603595  37.192113
18           1 -121.603513  37.192156
19           1 -121.603336  37.192229
20           1 -121.603154  37.192282
21           1 -121.603047  37.192309
22           1 -121.602962  37.192346
23           1 -121.602882  37.192397
24           1 -121.602820  37.192454
25          

In [9]:
# Check results on node Dataframe
print(node_df)

          id         lon        lat
0          0 -121.606408  37.191532
1          1 -121.601716  37.194956
2          2 -121.694674  37.234624
3          3 -121.694660  37.234351
4          4 -121.714070  37.207947
5          5 -121.713821  37.207574
6          6 -121.671492  37.171718
7          7 -121.669070  37.169351
8          8 -121.224514  37.078571
9          9 -121.229192  37.081465
10        10 -121.287002  37.090978
11        11 -121.289273  37.081942
12        12 -121.288139  37.093572
13        13 -121.225910  37.064720
14        14 -121.220900  37.065970
15        15 -121.213620  37.065230
16        16 -121.213180  37.065190
17        17 -121.233379  37.040413
18        18 -121.234821  37.037263
19        19 -121.643220  37.166830
20        20 -121.642530  37.166650
21        21 -121.673299  37.173286
22        22 -121.671692  37.171891
23        23 -121.321697  37.033036
24        24 -121.311920  37.035880
25        25 -121.282233  37.047108
26        26 -121.282233  37

In [10]:
# Check results on edge DataFrame
print(edge_df)

            id    src    dst  mtfcc
0            1      0      1      2
1            1      1      0      2
2            2      2      3      2
3            2      3      2      2
4            3      4      5      5
5            3      5      4      5
6            7      6      7      0
7            7      7      6      0
8           10      8      9      2
9           10      9      8      2
10          11     10     11      2
11          11     11     10      2
12          12     12     10      2
13          12     10     12      2
14          17     13     14      1
15          17     14     13      1
16          18     15     16      2
17          18     16     15      2
18          19     17     18      2
19          19     18     17      2
20          20     19     20      2
21          20     20     19      2
22          22     21     22      0
23          22     22     21      0
24          23     23     24      1
25          23     24     23      1
26          26     25     26

In [11]:
# Define chunk size and parameters
threads_per_block = 128
trunk_size = 10240
data_length = vector_df['id'].count()

In [12]:
# Haversine function for lengths
@cuda.jit(device=True)
def haversine_distance(lon_1, lat_1, lon_2, lat_2):
    lon_1 = lon_1 * math.pi / 180.0 
    lon_2 = lon_2 * math.pi / 180.0 
    lat_1 = lat_1 * math.pi / 180.0 
    lat_2 = lat_2 * math.pi / 180.0
    d_lon = lon_2 - lon_1 
    d_lat = lat_2 - lat_1 
    a = math.sin(d_lat/2)**2 + math.cos(lat_1) * math.cos(lat_2) * math.sin(d_lon/2)**2
    return 2 * math.asin(math.sqrt(a)) * 6371000.0

In [13]:
# Window function to calculate distance between adjacent points in feet
def adjacent_distance(id, lon, lat, dist):
    for i in range(cuda.threadIdx.x, id.size, cuda.blockDim.x):
        # If the first point of vector return zero length
        if i == 0:
            dist[i] = 0.0
        # Or if the vector changes (1st point of new vector)
        elif id[i] != id[i-1]:
            dist[i] = 0.0
        else:
            dist[i] = haversine_distance(lon[i],lat[i],lon[i-1],lat[i-1]) * 3.280839895013123

In [14]:
%%time
# Apply the window function
vector_df = vector_df.apply_chunks(adjacent_distance,
                     incols=['id','lon','lat'],
                     outcols=dict(dist=np.float64),
                     kwargs=dict(),
                     chunks=list(range(0, data_length,
                                       trunk_size))+ [data_length],
                     tpb=threads_per_block)

CPU times: user 449 ms, sys: 205 µs, total: 449 ms
Wall time: 448 ms


In [15]:
# Check the result
print(vector_df)

            id         lon        lat        dist
0            1 -121.606408  37.191532    0.000000
1            1 -121.606128  37.191692  100.142736
2            1 -121.605951  37.191783   61.221564
3            1 -121.605861  37.191814   28.495687
4            1 -121.605767  37.191832   28.096035
5            1 -121.605660  37.191836   31.130101
6            1 -121.605483  37.191840   51.459671
7            1 -121.605306  37.191857   51.811487
8            1 -121.605152  37.191881   45.603188
9            1 -121.604765  37.191944  114.792486
10           1 -121.604598  37.191971   49.522203
11           1 -121.604463  37.191984   39.518651
12           1 -121.604327  37.191989   39.565725
13           1 -121.604044  37.191988   82.244885
14           1 -121.603922  37.192002   35.821016
15           1 -121.603800  37.192030   36.897160
16           1 -121.603692  37.192068   34.311603
17           1 -121.603595  37.192113   32.621450
18           1 -121.603513  37.192156   28.530124


In [16]:
%%time
# Group by and sum the distances for each edge
distance_df = vector_df[['id','dist']].groupby(['id']).sum()
distance_df['id'] = distance_df.index

CPU times: user 90.4 ms, sys: 3.63 ms, total: 94 ms
Wall time: 92.8 ms


In [17]:
%%time
# Merge the distances into the edge DataFrame to produce graph
edge_df = edge_df.merge(distance_df,on=['id'], how='left').sort_values('id')

CPU times: user 18.9 ms, sys: 15.7 ms, total: 34.6 ms
Wall time: 33.2 ms


In [18]:
# Check Result
print(edge_df)

            id    src    dst  mtfcc         dist
52512        1      0      1      2  2129.795165
52513        1      1      0      2  2129.795165
52514        2      2      3      2    99.701199
52515        2      3      2      2    99.701199
52516        3      4      5      5   414.540894
52517        3      5      4      5   414.540894
52518        7      6      7      0  1114.588020
52519        7      7      6      0  1114.588020
52520       10      8      9      2  1801.042657
52521       10      9      8      2  1801.042657
52522       11     10     11      2  3362.048365
52523       11     11     10      2  3362.048365
52524       12     12     10      2  1170.219255
52525       12     10     12      2  1170.219255
52526       17     13     14      1  1529.094091
52527       17     14     13      1  1529.094091
52528       18     15     16      2   128.913876
52529       18     16     15      2   128.913876
52530       19     17     18      2  1284.591833
52531       19     1

In [19]:
# Save road graph to csv
graph_file = shp_name[:-4] + "_graph.csv"
edge_df.to_pandas().to_csv(graph_file, index=False)

In [20]:
# Save road modes to csv
node_file = shp_name[:-4] + "_nodes.csv"
node_df.to_pandas().to_csv(node_file, index=False)

In [21]:
# print mtfcc table (used to filter on road type)
print([(i,m) for i,m in enumerate(mtfcc)])

[(0, 'S1100'), (1, 'S1200'), (2, 'S1400'), (3, 'S1500'), (4, 'S1630'), (5, 'S1640'), (6, 'S1710'), (7, 'S1730'), (8, 'S1740'), (9, 'S1750'), (10, 'S1780'), (11, 'S1820')]
